<a href="https://colab.research.google.com/github/noambassat/SupremeCourtClassifier/blob/main/7_add_columns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import pandas as pd
import matplotlib.pyplot as plt
from pandas.api.types import CategoricalDtype
import ast
import joblib

In [ ]:
RAAP = pd.read_csv("RAAP.csv")

In [ ]:
full_ra_rap = pd.read_excel("full_ra_rap.xlsx")

In [ ]:
full_rap = full_ra_rap[full_ra_rap["סוג הליך"]=='רע"פ']
# בדיקה אם בעמודת "גוף המסמך" קיימת המילה "רע"פ"
full_rap = full_rap[full_rap["גוף המסמך"].str.contains('רע"פ', na=False)]

In [ ]:
full_rap

In [ ]:
full_rap.shape

In [ ]:
full_rap[["גוף המסמך"]].dropna(how='any', ignore_index=True,inplace=True)
full_rap[["מספר הליך","שם הליך"]].dropna(how='all', ignore_index=True,inplace=True)
full_rap[["מספר הליך","שם הליך"]].drop_duplicates(inplace=True, ignore_index=True)

In [ ]:
# פונקציה להמרת טקסט שמופיע כרשימה למחרוזת רגילה
def convert_list_to_string(text):
    if isinstance(text, str) and text.startswith("[") and text.endswith("]"):
        try:
            # מנסה להמיר את התוכן בתוך הסוגריים לרשימה אמיתית
            text_list = ast.literal_eval(text)
            # איחוד המחרוזות לרצף טקסט אחד
            return ' '.join(text_list)
        except (ValueError, SyntaxError):
            return text
    return text

# פונקציה לניקוי התווים המיותרים
def clean_text(text):
    if isinstance(text, list):
        text = ' '.join(text)  # הפיכת רשימה למחרוזת
    elif isinstance(text, str):
        # הסרת תווי רווח מיותרים ותווים מיוחדים
        text = re.sub(r'\n+', ' ', text)  # הסרת שורות חדשות מרובות
        text = re.sub(r'\\n', '', text)   # הסרת תווי newline \n מהטקסט
        text = re.sub(r'\\xa0', ' ', text)  # הסרת תווי \xa0 מהטקסט
        text = re.sub(r'\s+', ' ', text)  # הסרת רווחים מרובים
        return text.strip()
    return text

# הדפסת שורות לא קריאות לפני המרה
print("שורות לא קריאות לפני המרה:")
print(full_rap["גוף המסמך"].head())

# המרה של הטקסטים הלא קריאים לטקסטים קריאים ושמירה על הדאטה המקורי
full_rap["גוף המסמך"] = full_rap["גוף המסמך"].apply(clean_text)

print("\nשורות לאחר המרה:")
print(full_rap["גוף המסמך"].head())

file_path = 'full_rap_df_cleaned.csv'
full_rap.to_csv(file_path, index=False, encoding='utf-8')

# print(f"הקובץ נשמר בהצלחה בנתיב: {file_path}")


In [ ]:
full_rap

In [ ]:
full_final_df = pd.read_csv("full_final_df.csv")

In [ ]:
full_final_df.shape

In [ ]:
for col in RAAP.columns:
  print(col)

In [ ]:
sides = raap[[
"צד א'",
"צד ב'"
]]

In [ ]:
values = [val for value in sides.values for val in value]
values

In [ ]:
clean_values = set()
for val in values:
    val_str = str(val)  # המרה למחרוזת
    cleaned_val = re.sub(r"[\[\]'\\n]", "", val_str).strip()
    clean_values.add(cleaned_val)

In [ ]:
clean_values

In [ ]:
def check_sub(value,sub="מדינת ישראל"):
  if(value.find(sub)!=-1):
    return 1
  return 0

check_sub("מדינת ישראל – משרד התחבורה")

In [ ]:
new_sides = set()
for side in clean_values:
  if(check_sub(side)==0):
    new_sides.add(side)


In [ ]:
for side in new_sides:
  if(check_sub(side,"ישראל")==1):
    print(side)

In [ ]:
RAAP.shape

In [ ]:
ignore_df_1 = pd.read_excel("full_data_BOTH_models_with_predictions_V.xlsx")
ignore_df_1.dropna(subset=['בקשה לרשות ערעור התקבלה?'],inplace=True)

In [ ]:
ignore_df_2 = pd.read_excel("2_full_data_with_predictions V2.xlsx")
ignore_df_2.dropna(subset=['בקשה לרשות ערעור התקבלה?'],inplace=True)

In [ ]:
ignore_df_3 = pd.read_excel("full_data_with_predictions-VETTING.xlsx")
ignore_df_3.dropna(subset=['בקשה לרשות ערעור התקבלה?'],inplace=True)

In [ ]:
ignore_df = pd.concat([ignore_df_1,ignore_df_2,ignore_df_3])

In [ ]:
ignore_df.dropna(subset=['בקשה לרשות ערעור התקבלה?'],inplace=True)

In [ ]:
ignore_df.shape

In [ ]:
ignore_df.drop_duplicates(subset=['מספר הליך'],inplace=True)
ignore_df.shape

In [ ]:

raap = RAAP[~RAAP["מספר הליך"].isin(ignore_df["מספר הליך"])]

In [ ]:
raap.shape

In [ ]:
# פונקציה להמרת טקסט שמופיע כרשימה למחרוזת רגילה
def convert_list_to_string(text):
    if isinstance(text, str) and text.startswith("[") and text.endswith("]"):
        try:
            # מנסה להמיר את התוכן בתוך הסוגריים לרשימה אמיתית
            text_list = ast.literal_eval(text)
            # איחוד המחרוזות לרצף טקסט אחד
            return ' '.join(text_list)
        except (ValueError, SyntaxError):
            return text
    return text

# פונקציה לניקוי התווים המיותרים
def clean_text(text):
    if isinstance(text, list):
        text = ' '.join(text)  # הפיכת רשימה למחרוזת
    elif isinstance(text, str):
        # הסרת תווי רווח מיותרים ותווים מיוחדים
        text = re.sub(r'\n+', ' ', text)  # הסרת שורות חדשות מרובות
        text = re.sub(r'\\n', '', text)   # הסרת תווי newline \n מהטקסט
        text = re.sub(r'\\xa0', ' ', text)  # הסרת תווי \xa0 מהטקסט
        text = re.sub(r'\s+', ' ', text)  # הסרת רווחים מרובים
        return text.strip()
    return text

# הדפסת שורות לא קריאות לפני המרה
print("שורות לא קריאות לפני המרה:")
print(raap["גוף המסמך"].head())

# המרה של הטקסטים הלא קריאים לטקסטים קריאים ושמירה על הדאטה המקורי
raap["גוף המסמך"] = raap["גוף המסמך"].apply(clean_text)

print("\nשורות לאחר המרה:")
print(raap["גוף המסמך"].head())

# file_path = 'full_final_df_cleaned.csv'
# df_full_ra_rap.to_csv(file_path, index=False, encoding='utf-8')

# print(f"הקובץ נשמר בהצלחה בנתיב: {file_path}")
